In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Canal Óptico

O canal de comunicação óptica é o resultado de diversas interações entre sinais ópticos e matéria. Esses efeitos podem ocorrer nas diversas seções de fibra óptica do link ou nos dispositivos atravessados pelo sinal óptico. 

O efeito mais fundamental para qualquer sistema de comunicação é atenuação, que reduz a potência do sinal ao longo do link como resultado de fenômenos de absorção e espalhamento. Devido à atenuação, os sinais transmitidos precisam ser amplificados periodicamente por amplificadores em linha, que inserem ruído e limitam o alcance do sistema.

Neste notebook não iremos abordar todos esses efeitos, mas apenas aqueles mais significativos para a modelagem de sistemas ópticos digitais coerentes.

### 3.2 Chromatic Dispersion

A dispersão do modo de polarização (*Polarization mode dispersion* - PMD) e a dispersão cromática (*Chromatic Dispersion* - CD) são dois efeitos lineares importantes que espalham símbolos ao longo do tempo e causam interferência. Eles geralmente são compensados no receptor usando usando tecnicas de processamento digital de sinais (DSP) Embora o PMD e o CD individualmente não limitem a capacidade, eles influenciam a complexidade do DSP do receptor, pois o CD pode exigir equalizadores estáticos muito longos e o PMD exige equalização adaptativa.

Neste tópico estamos interessados em transmitir informações através de fibras ópticas. A informação é transmitida apenas por pacotes de frequências, formando um padrão que viaja pela fibra com velocidade de grupo $v_g$

$$v_{g}=\left({\frac{d\beta}{d\omega}}\right)^{-1}$$

onde $\beta$ é a constante de propagação. Outro conceito necessário para entendermos esse fenômeno é a velocidade de fase $v_p$ de um sinal senoidal com frequência $\omega$ dado por

$$\ v_{p}={\frac{\omega}{\beta}}$$

Se a constante de propagação tiver uma dependência linear com a frequência, então $v_g = v_p$, qualquer grupo de frequências de propagação terá a mesma velocidade de grupo e os pulsos não serão distorcidos. É o caso, por exemplo, das ondas planas que viajam no espaço livre. Entretanto, em guias de onda práticos, a constante de propagação $\beta$ tem uma dependência não linear de $\omega$. Assim, diferentes componentes espectrais de um modo guiado se propagam com velocidades de grupo distintas, dando origem ao fenômeno conhecido como CD.

Se não for compensado, o CD amplia os pulsos de propagação e leva à interferência intersimbólica (ISI), como mostrado na figura abaixo que demostra o efeito do CD na propagação do sinal. CD espalha os pulsos no tempo, levando à interferência intersimbólica (ISI)

<br>
<center>
    <img src="https://i.postimg.cc/6pYm6pmx/Dispers-o.png">
</center>
<br>

